In [30]:
import cv2
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt

# handle exception trace for debugging 
# background loop
import traceback

import time
import random

In [31]:
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice())
cam.Open()

# to get consistant results it is always good to start from "power-on" state
cam.UserSetSelector.Value = "Default"
cam.UserSetLoad.Execute()

cam.ExposureTime.Value = cam.ExposureTime.Min

In [4]:
cam.ExposureTime.Min

19.0

In [5]:
# show expected framerate max framerate ( @ minimum exposure time)
cam.ResultingFrameRate.Value

156.25

In [6]:
# this results in frame period in µs
1 / cam.ResultingFrameRate.Value * 1e6

6400.0

we now compare different pylon grab strategies and their resulting frameperiod to this minimum frameperiod

# Grab scenarios

## GrabOne Loop

most simple style to grab

In [72]:
def GrabOneSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)

    for i in range(100):
        with cam.GrabOne(1000) as res:
            img = res.Array
            img_sum += img
    return img_sum

In [ ]:
%%timeit -o
GrabOneSample()

In [16]:
grab_one_average = _.average / 100

In [24]:
f"time to capture one frame is: {grab_one_average*1e6:.2f} µs"

'time to capture one frame is: 151589.75 µs'

this is very easy to use, but will have the overhead of starting and stopping the grab engine for every frame

## Foreground Loop

move the start and stop of the grab engine out of the inner grab loop

In [7]:
def ForegroundLoopSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
    cam.StartGrabbingMax(100)
    while cam.IsGrabbing():
        with cam.RetrieveResult(1000) as res:
            if res.GrabSucceeded():
                img = res.Array
                img_sum += img
            else:
                raise RuntimeError("Grab failed")
    cam.StopGrabbing()
    return img_sum

In [10]:
img_sum = ForegroundLoopSample()

(1920,)

In [12]:
import matplotlib.pyplot as plt


In [26]:
%%timeit -o 
ForegroundLoopSample()

713 ms ± 61.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 713 ms ± 61.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [27]:
foreground_average = _.average / 100

In [28]:
f"average time to capture one frame is: {foreground_average*1e6:.2f} µs"

'average time to capture one frame is: 7125.70 µs'

## Background loop

* use this mode to communicate with the camera while the images are collected in the backgound.
* Allows async communication

Pylon uses classes with callback handler to let usercode communicate with the grab loop

In [29]:
class ImageHandler(py.ImageEventHandler):
    def __init__(self):
        super().__init__()
        self.img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
        
    def OnImageGrabbed(self, camera, grabResult):
        """ we get called on every image
            !! this code is run in a pylon thread context
            always wrap your code in the try .. except to capture
            errors inside the grabbing as this can't be properly reported from 
            the background thread to the foreground python code
        """
        try:
            if grabResult.GrabSucceeded():
                # check image contents
                img = grabResult.Array
                self.img_sum += img
            else:
                raise RuntimeError("Grab Failed")
        except Exception as e:
            traceback.print_exc()

In [37]:
def BackGroundLoopSample():

    # instantiate callback handler
    handler = ImageHandler()
    # register with the pylon loop
    cam.RegisterImageEventHandler(handler , py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # fetch some images with background loop
    cam.StartGrabbingMax(100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    while cam.IsGrabbing():
        # random exposuretime changes every 100ms
        # cam.ExposureTime.Value = random.uniform(cam.ExposureTime.Min, 1000)
        time.sleep(0.1)

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)

    return handler.img_sum

In [38]:
%%timeit -o
BackGroundLoopSample()

711 ms ± 443 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 711 ms ± 443 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [40]:
background_average = _.average / 100

In [41]:
f"average time to capture one frame is: {background_average*1e6:.2f} µs"

'average time to capture one frame is: 7105.98 µs'

In [ ]:
cam.StopGrabbing()

In [9]:
cam.Close()

In [56]:
cam.DeviceTemperatureSelector.SetValue("Coreboard")
print(cam.DeviceTemperature.Value)
cam.BslSensorOn.Execute()
print(cam.BslSensorState.Value)
cam.DeviceTemperatureSelector.SetValue("Sensor")
print(cam.DeviceTemperature.Value)
cam.BslSensorStandby.Execute()
print(cam.BslSensorState.Value)

65.875
On
58.74400000000003
Standby


67.25

In [66]:
time.time() * 1000000

1756233142039616.0

50000


In [1]:
from pypylon import pylon
from pypylon import genicam
import time

import sys


# Example of a device specific handler for image events.
class SampleImageEventHandler(pylon.ImageEventHandler):
    def OnImageGrabbed(self, camera, grabResult):
        # The chunk data is attached to the grab result and can be accessed anywhere.

        # Native parameter access:
        # When using the device specific grab results the chunk data can be accessed
        # via the members of the grab result data.
        if genicam.IsReadable(grabResult.ChunkTimestamp):
            print("OnImageGrabbed: TimeStamp (Result) accessed via result member: ", grabResult.ChunkTimestamp.Value)


# Number of images to be grabbed.
countOfImagesToGrab = 100

# The exit code of the sample application.
exitCode = 0

try:
    # Only look for cameras supported by Camera_t
    info = pylon.DeviceInfo()
    info.SetDeviceClass("BaslerUsb")

    # Create an instant camera object with the first found camera device that matches the specified device class.
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice(info))
    di = camera.GetDeviceInfo()
    print(di.GetDeviceClass())

    # Print the model name of the camera.
    print("Using device ", camera.GetDeviceInfo().GetModelName())

    # Register an image event handler that accesses the chunk data.
    camera.RegisterImageEventHandler(SampleImageEventHandler(), pylon.RegistrationMode_Append, pylon.Cleanup_Delete)

    # Open the camera.
    camera.Open()

    # A GenICam node map is required for accessing chunk data. That's why a small node map is required for each grab result.
    # Creating a node map can be time consuming, because node maps are created by parsing an XML description file.
    # The node maps are usually created dynamically when StartGrabbing() is called.
    # To avoid a delay caused by node map creation in StartGrabbing() you have the option to create
    # a static pool of node maps once before grabbing.
    camera.StaticChunkNodeMapPoolSize.Value = camera.MaxNumBuffer.GetValue()

    # Enable chunks in general.
    if genicam.IsWritable(camera.ChunkModeActive):
        camera.ChunkModeActive.Value = True
    else:
        raise pylon.RuntimeException("The camera doesn't support chunk features")

    # Enable time stamp chunks.
    camera.ChunkSelector.Value = "Timestamp"
    camera.ChunkEnable.Value = True

    if not camera.IsUsb():
        # Enable frame counter chunks.
        camera.ChunkSelector.Value = "Framecounter"
        camera.ChunkEnable.Value = True

    # Enable CRC checksum chunks.
    camera.ChunkSelector.Value = "PayloadCRC16"
    camera.ChunkEnable.Value = True

    # Start the grabbing of c_countOfImagesToGrab images.
    # The camera device is parameterized with a default configuration which
    # sets up free-running continuous acquisition.
    # camera.StartGrabbingMax(countOfImagesToGrab)
    camera.StartGrabbing(True)
    ###------------------------------####
    update_time = 1
    frame_count = 0
    start_time = time.time()
    previous_time = time.time()

    previous_time_2 = 0


    counter = 0
    # Camera.StopGrabbing() is called automatically by the RetrieveResult() method
    # when c_countOfImagesToGrab images have been retrieved.
    while camera.IsGrabbing():
        counter += 1
        print(counter)
        # Wait for an image and then retrieve it. A timeout of 5000 ms is used.
        # RetrieveResult calls the image event handler's OnImageGrabbed method.
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
        print("Grabbed ", grabResult.ChunkData.Value)

        print("GrabSucceeded: ", grabResult.GrabSucceeded())

        print(f"fps cam {1 / ((grabResult.ChunkTimestamp.Value - previous_time_2) / 1000000000)}")
        previous_time_2 = grabResult.ChunkTimestamp.Value

        frame_count += 1
        TIME = time.time() - previous_time

        if (TIME) >= update_time:
            # Calculate the FPS value
            FPS = frame_count / (TIME)
            frame_count = 0
            previous_time = time.time()
            fps_disp = f"FPS: {FPS:.2f}"
            print(fps_disp)

        # The result data is automatically filled with received chunk data.
        # (Note:  This is not the case when using the low-level API)
        print("SizeX: ", grabResult.Width)
        print("SizeY: ", grabResult.Height)
        img = grabResult.GetArray()
        print("Gray value of first pixel: ", img[0, 0])

        # Check to see if a buffer containing chunk data has been received.
        if pylon.PayloadType_ChunkData != grabResult.PayloadType:
            raise pylon.RuntimeException("Unexpected payload type received.")

        # Since we have activated the CRC Checksum feature, we can check
        # the integrity of the buffer first.
        # Note: Enabling the CRC Checksum feature is not a prerequisite for using
        # chunks. Chunks can also be handled when the CRC Checksum feature is deactivated.
        if grabResult.HasCRC() and grabResult.CheckCRC() == False:
            raise pylon.RuntimeException("Image was damaged!")

        # Access the chunk data attached to the result.
        # Before accessing the chunk data, you should check to see
        # if the chunk is readable. When it is readable, the buffer
        # contains the requested chunk data.
        if genicam.IsReadable(grabResult.ChunkTimestamp):
            print("TimeStamp (Result): ", grabResult.ChunkTimestamp.Value)
            print(time.time())

        # USB camera devices provide generic counters. An explicit FrameCounter value is not provided by USB camera devices.
        if not camera.IsUsb():
            if genicam.IsReadable(grabResult.ChunkFramecounter):
                print("FrameCounter (Result): ", grabResult.ChunkFramecounter.Value)
        print()

    # Disable chunk mode.
    camera.ChunkModeActive.Value = False
    camera.Close()

except genicam.GenericException as e:
    # Error handling.
    print("An exception occurred.", e)
    exitCode = 1

BaslerUsb
Using device  a2A1920-160ucPRO
1
OnImageGrabbed: TimeStamp (Result) accessed via result member:  5367768568550


AttributeError: no attribute 'ChunkData' in GrabResult